### 최종 코드 (pymysql + crawling)

In [6]:
import requests
from bs4 import BeautifulSoup
import pymysql

In [20]:
db = pymysql.connect(host='localhost', port=3306, user='root', passwd='3927', db='bestproducts', charset='utf8')
cursor = db.cursor()

res = requests.get('http://corners.gmarket.co.kr/Bestsellers')
soup = BeautifulSoup(res.content, 'html.parser')

categories = soup.select('div.gbest-cate ul.by-group li a')
# for category in categories:
#     get_category('http://corners.gmarket.co.kr/' + category['href'], category.get_text())

In [3]:
res = requests.get('http://corners.gmarket.co.kr/' + categories[0]['href'], categories[0].get_text())
soup = BeautifulSoup(res.content, 'html.parser')

In [22]:
soup.select('div.best-list')

[<div class="best-list type2" id="topPlusItems">
 <ul>
 <li>
 <p class="ico_plus">PLUS</p>
 <div class="thumb">
 <a href="" id="topPlusItemLink0"><img height="150" id="topPlusItemImage0" src="http://image.gmarket.co.kr/challenge/neo_image/no_image.gif" width="150"/></a>
 </div>
 <a class="itemname" href="" id="topPlusItemName0"></a>
 <div class="o-price" id="topPlusItemPrice0">\</div>
 <div class="s-price">
 <strong id="topPlusItemDiscountPrice0">\</strong>
 <span><span class="sale" id="downArrowIcon0"></span><em id="topPlusItemDiscountRate0"></em></span>
 </div>
 </li>
 <li>
 <p class="ico_plus">PLUS</p>
 <div class="thumb">
 <a href="" id="topPlusItemLink1"><img height="150" id="topPlusItemImage1" src="http://image.gmarket.co.kr/challenge/neo_image/no_image.gif" width="150"/></a>
 </div>
 <a class="itemname" href="" id="topPlusItemName1"></a>
 <div class="o-price" id="topPlusItemPrice1">\</div>
 <div class="s-price">
 <strong id="topPlusItemDiscountPrice1">\</strong>
 <span><span cla

In [4]:
html = soup
category_name = categories[0].get_text()
subcategory_name = 'ALL'

best_item = html.select('div.best-list')

In [2]:
## 페이지 구조
## 전체ALL -> Main Category -> Main Category의 ALL -> subCategory

#################################################################################################
# main category list parsing
def get_category(category_link, category_name): # 메인 카테고리링크, 메인카테고리명
#     print(category_link, category_name)
    res = requests.get(category_link) # 해당 메인카테고리 페이지로
    soup = BeautifulSoup(res.content, 'html.parser')

    # get_items가 여기서 한번 아래에서 다시 한번(for문으로) 총 2번 실행된다
    # 여기서는 각 main카테고리의 main 정보들을 parsing 한다.
    get_items(soup, category_name, "ALL") # 여기서 ALL은 그냥 이름 지정위해 사용
    
    # 각 main 카테고리에서의 sub category 정보들을 파싱한다.
    sub_categories = soup.select('div.navi.group ul li > a') # li 바로 밑에 a태그가 있는것만 추출
    for sub_category in sub_categories:
        res = requests.get('http://corners.gmarket.co.kr/' + sub_category['href'])
        soup = BeautifulSoup(res.content, 'html.parser')
        print(category_name, sub_category.get_text())
        get_items(soup, category_name, sub_category.get_text()) # 각 sub_category 별로 정보 파싱
        
#################################################################################################

# 정보 파싱하여 dict 형태로 저장
def get_items(html, category_name, sub_category_name):
    best_item = html.select('div.best-list')
    # div.best-list tag가 여러개임 그 중 2번째가 필요
    for index, item in enumerate(best_item[1].select('li')):
        # 각각의 li별로 정보 수집하여 dict에 저장
        data_dict = dict()

        ranking = index + 1 
        title = item.select_one('a.itemname')
        ori_price = item.select_one('div.o-price')
        dis_price = item.select_one('div.s-price strong span') # storng sapn태그까지 들어가야 정확.
        discount_percent = item.select_one('div.s-price em')
        
        # ori_price와 dis_pirce가 없는 경우가 존재함.
        if ori_price == None or ori_price.get_text() == '': # 태그가 아예 없는 경우 or 태그는 있으나 정보가 비어있는 경우
            ori_price = dis_price

        if dis_price == None:
            ori_price, dis_price = 0, 0
        else:
            ori_price = ori_price.get_text().replace(',', '').replace('원', '')
            dis_price = dis_price.get_text().replace(',', '').replace('원', '')

        if discount_percent == None or discount_percent.get_text() == '':
            discount_percent = 0
        else:
            discount_percent = discount_percent.get_text().replace('%', '')
        
        product_link = item.select_one('div.thumb > a') # 각 상품별 페이지 링크
        item_code = product_link.attrs['href'].split('=')[1] # 상품 id만 추출

        res = requests.get(product_link.attrs['href']) # 상품별 페이지로 
        soup = BeautifulSoup(res.content, 'html.parser')
        provider = soup.select_one('div.item-topinfo_headline > p > a > strong') # 판매자 파싱
        if provider == None:
            provider = ''
        else:
            provider = provider.get_text()

        data_dict['category_name'] = category_name
        data_dict['sub_category_name'] = sub_category_name
        data_dict['ranking'] = ranking
        data_dict['title'] = title.get_text()
        data_dict['ori_price'] = ori_price
        data_dict['dis_price'] = dis_price
        data_dict['discount_percent'] = discount_percent
        data_dict['item_code'] = item_code
        data_dict['provider'] = provider

        save_data(data_dict) # db에 insert
        
#################################################################################################


# Main에서도 순위권이고 sub에서도 순위권인 상품의 경우 item_code가 중복됨. 이 경우 고려 
def save_data(item_info):
#     print (item_info)
    
    # 이미 존재하는 item_code인지 확인
    sql = """SELECT COUNT(*) FROM items WHERE item_code = '""" + item_info['item_code'] + """';"""
    cursor.execute(sql)
    result = cursor.fetchone()
    
    if result[0] == 0: # 새로들어오는 item_code라면 
        sql = """INSERT INTO items VALUES('""" + item_info['item_code'] + """',
        '""" + item_info['title'] + """', 
        """ + str(item_info['ori_price']) + """, 
        """ + str(item_info['dis_price']) + """, 
        """ + str(item_info['discount_percent']) + """, 
        '""" + item_info['provider'] + """')"""
        print (sql)
        cursor.execute(sql)
    
    # item_code가 이미 존재한다면 여기만 실행
    sql = """INSERT INTO ranking (main_category, sub_category, item_ranking, item_code) VALUES('""" + item_info['category_name'] + """',
    '""" + item_info['sub_category_name'] + """', 
    '""" + str(item_info['ranking']) + """', 
    '""" + item_info['item_code'] + """')"""     
    print (sql)    
    cursor.execute(sql)
    
#################################################################################################

db = pymysql.connect(host='localhost', port=3306, user='root', passwd='3927', db='bestproducts', charset='utf8')
cursor = db.cursor()

res = requests.get('http://corners.gmarket.co.kr/Bestsellers')
soup = BeautifulSoup(res.content, 'html.parser')

categories = soup.select('div.gbest-cate ul.by-group li a')
for category in categories:
    get_category('http://corners.gmarket.co.kr/' + category['href'], category.get_text())

db.commit()
db.close()

In [ ]:
# ==== insert example ====
sql = """insert into customer(name,category,region)
         values (%d, %d, %d)"""
curs.execute(sql, ('홍길동', 1, '서울'))
conn.commit()

sql = """
INSERT INTO corona_hospital(lat,lng) VALUES (%f,%f) WHERE id = %f
"""

